In [1]:
import numpy as np

http://www.scipy-lectures.org/advanced/optimizing/index.html
    
https://wiki.python.org/moin/PythonSpeed/PerformanceTips


In [2]:
a = np.arange(1000)


%timeit a**2



The slowest run took 5502.65 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.27 µs per loop
